In [1]:
import pandas as pd
import os
import time

In [2]:
import joblib
from joblib import Parallel,delayed

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
stock_basic = pd.read_csv("stock_basic.csv")
stock_basic

,ts_code,symbol,name,area,industry,list_date
0,000001.SZ,1,平安银行,深圳,银行,19910403
1,000002.SZ,2,万科A,深圳,全国地产,19910129
2,000004.SZ,4,国华网安,深圳,软件服务,19910114
3,000006.SZ,6,深振业A,深圳,区域地产,19920427
4,000007.SZ,7,全新好,深圳,其他商业,19920413
...,...,...,...,...,...,...
4530,605580.SH,605580,恒盛能源,浙江,供气供热,20210819
4531,605588.SH,605588,冠石科技,江苏,元器件,20210812
4532,605589.SH,605589,圣泉集团,山东,化工原料,20210810
4533,605598.SH,605598,上海港湾,上海,建筑工程,20210917


In [5]:
trade_cal=pd.read_csv("trade_cal.csv")
trade_cal

,exchange,cal_date,is_open,pretrade_date
0,SSE,20141027,1,20141024
1,SSE,20141028,1,20141027
2,SSE,20141029,1,20141028
3,SSE,20141030,1,20141029
4,SSE,20141031,1,20141030
...,...,...,...,...
2427,SSE,20241021,1,20241018
2428,SSE,20241022,1,20241021
2429,SSE,20241023,1,20241022
2430,SSE,20241024,1,20241023


In [6]:
trade_cal_st = trade_cal.copy()
trade_cal_st["index"]=trade_cal_st.index.tolist()
trade_cal_st.index=trade_cal_st.cal_date.tolist()
trade_cal_st

,exchange,cal_date,is_open,pretrade_date,index
20141027,SSE,20141027,1,20141024,0
20141028,SSE,20141028,1,20141027,1
20141029,SSE,20141029,1,20141028,2
20141030,SSE,20141030,1,20141029,3
20141031,SSE,20141031,1,20141030,4
...,...,...,...,...,...
20241021,SSE,20241021,1,20241018,2427
20241022,SSE,20241022,1,20241021,2428
20241023,SSE,20241023,1,20241022,2429
20241024,SSE,20241024,1,20241023,2430


In [7]:
custom_date = pd.read_csv("custom_date.csv")
start_date=str(custom_date.start_date[0])
end_date=str(custom_date.end_date[0])
print(start_date)
print(end_date)

20141027
20241027


In [8]:
if not os.path.isdir("stock_pool"):
    os.makedirs("stock_pool")
    print("已创建stock_pool文件夹")
else:
    print("stock_pool文件夹已存在")

stock_pool文件夹已存在


In [9]:
all_index = stock_basic.shape[0]
df_index = pd.to_datetime(trade_cal.cal_date.tolist(),format='%Y%m%d')

# 60天停牌

In [10]:
#停牌
# path = "stock_pool/suspend"
# stock_name = stock_basic.ts_code[9]
# df=pd.read_csv(path+"/"+stock_name+".csv")
# condition = df.suspend_type=="S"
# df=df[condition]
# df 

In [11]:
path = "stock_pool/suspend"
@joblib.delayed
def task(stock_name):
    current_df=pd.read_csv(path+"/"+stock_name+".csv")
    condition = current_df.suspend_type=="S"
    current_df=current_df[condition]
    new_df = pd.DataFrame(columns=[stock_name],index = trade_cal.index.tolist())
    for index, row in current_df.iterrows():
        new_df[stock_name][index:index+60]=1               
    return new_df

tasks=[]

for index, row in stock_basic.iterrows():
    stock_name = row.ts_code
#     print(stock_name)
    tasks.append(task(stock_name))
    
df = pd.concat(
    joblib.Parallel(n_jobs=5, verbose=10)(
        tasks
    ), axis=1)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   3 tasks      | elapsed:    1.4s
[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:    1.4s
[Parallel(n_jobs=5)]: Done  15 tasks      | elapsed:    1.5s
[Parallel(n_jobs=5)]: Batch computation too fast (0.1821s.) Setting batch_size=2.
[Parallel(n_jobs=5)]: Done  22 tasks      | elapsed:    1.5s
[Parallel(n_jobs=5)]: Done  32 tasks      | elapsed:    1.6s
[Parallel(n_jobs=5)]: Batch computation too fast (0.1237s.) Setting batch_size=4.
[Parallel(n_jobs=5)]: Done  50 tasks      | elapsed:    1.7s
[Parallel(n_jobs=5)]: Batch computation too fast (0.1371s.) Setting batch_size=8.
[Parallel(n_jobs=5)]: Done  98 tasks      | elapsed:    2.1s
[Parallel(n_jobs=5)]: Done 186 tasks      | elapsed:    2.6s
[Parallel(n_jobs=5)]: Done 290 tasks      | elapsed:    3.2s
[Parallel(n_jobs=5)]: Done 394 tasks      | elapsed:    3.8s
[Parallel(n_jobs=5)]: Done 514 tasks      | elapsed:    4.2s
[Paralle

In [12]:
df.fillna(0, inplace=True)
df

,000001.SZ,000002.SZ,000004.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,000011.SZ,000012.SZ,...,605507.SH,605555.SH,605566.SH,605567.SH,605577.SH,605580.SH,605588.SH,605589.SH,605598.SH,605599.SH
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2427,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2428,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2429,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2430,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# (df['002624.SZ'] == 1).sum()

In [14]:
df.to_csv("suspend.csv",index=0)

# 60天ST

In [15]:
#ST
# path = "stock_pool/st"
# stock_name = stock_basic.ts_code[100]
# df=pd.read_csv(path+"/"+stock_name+".csv")
# df=df[df['name'].str.contains("ST")]
# df.dropna(inplace=True)
# df.reset_index(inplace=True,drop=True)
# df

In [16]:
path = "stock_pool/st"
min_date = trade_cal_st.index[0]
max_date = trade_cal_st.index[trade_cal_st.shape[0]-1]
@joblib.delayed
def task(stock_name):
    current_df=pd.read_csv(path+"/"+stock_name+".csv")
    current_df=current_df[current_df['name'].str.contains("ST")]
    current_df.dropna(inplace=True)
    current_df.reset_index(inplace=True,drop=True)
    new_df = pd.DataFrame(columns=[stock_name],index = trade_cal.index.tolist())
    for index, row in current_df.iterrows():
        st_start_date = int(row.start_date)
        st_end_date = int(row.end_date)
        if st_start_date<min_date:
            st_start_date=min_date
        if st_start_date>max_date:
            return new_df
        if st_end_date<min_date:
            return new_df
        if st_end_date>max_date:
            st_end_date=max_date
        if st_start_date not in trade_cal_st.index:
            st_start_date = pd.to_datetime(st_start_date,format='%Y%m%d')
            st_start_date = int((st_start_date+pd.Timedelta(days=-1)).strftime('%Y%m%d'))
        if st_end_date not in trade_cal_st.index:
            st_end_date = pd.to_datetime(st_end_date,format='%Y%m%d')
            st_end_date = int((st_end_date+pd.Timedelta(days=1)).strftime('%Y%m%d'))
        new_df[stock_name][trade_cal_st["index"][st_start_date]:trade_cal_st["index"][st_end_date]+60]=1               
    return new_df

tasks=[]

for index, row in stock_basic.iterrows():
    stock_name = row.ts_code
#     print(stock_name)
    tasks.append(task(stock_name))
    
df = pd.concat(
    joblib.Parallel(n_jobs=5, verbose=10)(
        tasks
    ), axis=1)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Batch computation too fast (0.0170s.) Setting batch_size=2.
[Parallel(n_jobs=5)]: Done   3 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Batch computation too fast (0.0344s.) Setting batch_size=4.
[Parallel(n_jobs=5)]: Done  20 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Batch computation too fast (0.0688s.) Setting batch_size=8.
[Parallel(n_jobs=5)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done  78 tasks      | elapsed:    0.1s
[Parallel(n_jobs=5)]: Batch computation too fast (0.1274s.) Setting batch_size=16.
[Parallel(n_jobs=5)]: Done 150 tasks      | elapsed:    0.3s
[Parallel(n_jobs=5)]: Done 326 tasks      | elapsed:    0.7s
[Parallel(n_jobs=5)]: Done 502 tasks      | elapsed:    1.1s
[Parallel(n_jobs=5)]: Done 710 tasks      | elapsed:    1.4s
[Parallel(n_jobs=5)]: Done 918 tasks      | ela

In [17]:
df.fillna(0, inplace=True)
df

,000001.SZ,000002.SZ,000004.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,000011.SZ,000012.SZ,...,605507.SH,605555.SH,605566.SH,605567.SH,605577.SH,605580.SH,605588.SH,605589.SH,605598.SH,605599.SH
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2427,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2428,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2429,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2430,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
df.to_csv("st.csv",index=0)

In [19]:
# (df['002624.SZ'] == 1).sum()

In [20]:
# trade_cal_st

In [21]:
# int(df.start_date[0]) in trade_cal_st.index

In [22]:
# trade_cal_st["index"][int(df.start_date[0])]

In [23]:
# trade_cal_st["index"][20210301]

In [24]:
# current_date = pd.to_datetime(20210301,format='%Y%m%d')
# int((current_date+pd.Timedelta(days=1)).strftime('%Y%m%d'))

In [25]:
# df['name'].str.contains('ST').any()

# 不是ST

In [26]:
# path = "stock_pool/st"
# min_date = trade_cal_st.index[0]
# max_date = trade_cal_st.index[trade_cal_st.shape[0]-1]
# @joblib.delayed
# def task(stock_name):
#     current_df=pd.read_csv(path+"/"+stock_name+".csv")
#     current_df=current_df[current_df['name'].str.contains("ST")]
#     current_df.dropna(inplace=True)
#     current_df.reset_index(inplace=True,drop=True)
#     new_df = pd.DataFrame(columns=[stock_name],index = trade_cal.index.tolist())
#     for index, row in current_df.iterrows():
#         st_start_date = int(row.start_date)
#         st_end_date = int(row.end_date)
#         if st_start_date<min_date:
#             st_start_date=min_date
#         if st_start_date>max_date:
#             return new_df
#         if st_end_date<min_date:
#             return new_df
#         if st_end_date>max_date:
#             st_end_date=max_date
#         if st_start_date not in trade_cal_st.index:
#             st_start_date = pd.to_datetime(st_start_date,format='%Y%m%d')
#             st_start_date = int((st_start_date+pd.Timedelta(days=-1)).strftime('%Y%m%d'))
#         if st_end_date not in trade_cal_st.index:
#             st_end_date = pd.to_datetime(st_end_date,format='%Y%m%d')
#             st_end_date = int((st_end_date+pd.Timedelta(days=1)).strftime('%Y%m%d'))
#         new_df[stock_name][trade_cal_st["index"][st_start_date]:trade_cal_st["index"][st_end_date]]=0              
#     return new_df

# tasks=[]

# for index, row in stock_basic.iterrows():
#     stock_name = row.ts_code
# #     print(stock_name)
#     tasks.append(task(stock_name))
    
# df = pd.concat(
#     joblib.Parallel(n_jobs=5, verbose=10)(
#         tasks
#     ), axis=1)

In [27]:
# df.fillna(1, inplace=True)
# df

In [28]:
# df.to_csv("not_st.csv",index=0)

# 尾盘涨停

In [29]:
# path = "stock_pool/limit"
# stock_name = "002624.SZ"
# current_df=pd.read_csv(path+"/"+stock_name+".csv")
# current_stock_daily = pd.read_csv("stock_daily/"+stock_name+".csv")
# current_df["close"]=current_stock_daily["close"]
# current_df[stock_name]=(abs(current_df["up_limit"]-current_df["close"])<=0.01).astype(int)
# current_df=current_df[[stock_name]]
# current_df

In [30]:
# current_df[stock_name].isin([1]).any()

In [31]:
# current_stock_daily

In [32]:
path = "stock_pool/limit"
@joblib.delayed
def task(stock_name):
    current_df=pd.read_csv(path+"/"+stock_name+".csv")
    current_stock_daily = pd.read_csv("stock_daily/"+stock_name+".csv")
    current_df["close"]=current_stock_daily["close"]
    current_df[stock_name]=(abs(current_df["up_limit"]-current_df["close"])<0.01).astype(int)
    current_df=current_df[[stock_name]]         
    return current_df

tasks=[]

for index, row in stock_basic.iterrows():
    stock_name = row.ts_code
#     print(stock_name)
    tasks.append(task(stock_name))
    
df = pd.concat(
    joblib.Parallel(n_jobs=5, verbose=10)(
        tasks
    ), axis=1)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Batch computation too fast (0.0339s.) Setting batch_size=2.
[Parallel(n_jobs=5)]: Done   3 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Batch computation too fast (0.0933s.) Setting batch_size=4.
[Parallel(n_jobs=5)]: Done  20 tasks      | elapsed:    0.1s
[Parallel(n_jobs=5)]: Batch computation too fast (0.1661s.) Setting batch_size=8.
[Parallel(n_jobs=5)]: Done  38 tasks      | elapsed:    0.2s
[Parallel(n_jobs=5)]: Done  78 tasks      | elapsed:    0.6s
[Parallel(n_jobs=5)]: Done 150 tasks      | elapsed:    1.0s
[Parallel(n_jobs=5)]: Done 238 tasks      | elapsed:    1.6s
[Parallel(n_jobs=5)]: Done 326 tasks      | elapsed:    2.2s
[Parallel(n_jobs=5)]: Done 430 tasks      | elapsed:    2.8s
[Parallel(n_jobs=5)]: Done 534 tasks      | elapsed:    3.5s
[Parallel(n_jobs=5)]: Done 654 tasks      | elapsed:    4.3s
[Paralle

In [33]:
df

,000001.SZ,000002.SZ,000004.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,000011.SZ,000012.SZ,...,605507.SH,605555.SH,605566.SH,605567.SH,605577.SH,605580.SH,605588.SH,605589.SH,605598.SH,605599.SH
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2427,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2428,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2429,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2430,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
df.to_csv("limit.csv",index=0)

# 新股

In [35]:
# 新股
stock_basic

,ts_code,symbol,name,area,industry,list_date
0,000001.SZ,1,平安银行,深圳,银行,19910403
1,000002.SZ,2,万科A,深圳,全国地产,19910129
2,000004.SZ,4,国华网安,深圳,软件服务,19910114
3,000006.SZ,6,深振业A,深圳,区域地产,19920427
4,000007.SZ,7,全新好,深圳,其他商业,19920413
...,...,...,...,...,...,...
4530,605580.SH,605580,恒盛能源,浙江,供气供热,20210819
4531,605588.SH,605588,冠石科技,江苏,元器件,20210812
4532,605589.SH,605589,圣泉集团,山东,化工原料,20210810
4533,605598.SH,605598,上海港湾,上海,建筑工程,20210917


In [36]:
stock_basic["is_new"]=int(end_date)-stock_basic.list_date<20000# 小于两年为新股 为 True
stock_basic

,ts_code,symbol,name,area,industry,list_date,is_new
0,000001.SZ,1,平安银行,深圳,银行,19910403,False
1,000002.SZ,2,万科A,深圳,全国地产,19910129,False
2,000004.SZ,4,国华网安,深圳,软件服务,19910114,False
3,000006.SZ,6,深振业A,深圳,区域地产,19920427,False
4,000007.SZ,7,全新好,深圳,其他商业,19920413,False
...,...,...,...,...,...,...,...
4530,605580.SH,605580,恒盛能源,浙江,供气供热,20210819,False
4531,605588.SH,605588,冠石科技,江苏,元器件,20210812,False
4532,605589.SH,605589,圣泉集团,山东,化工原料,20210810,False
4533,605598.SH,605598,上海港湾,上海,建筑工程,20210917,False


In [37]:
@joblib.delayed
def task(stock_name,is_new):
    current_df=pd.DataFrame(index= trade_cal.index.tolist(),columns=[stock_name])
    current_df[stock_name]=is_new.astype(int)
    return current_df

tasks=[]

for index, row in stock_basic.iterrows():
    stock_name = stock_basic.ts_code[index]
    is_new = stock_basic.is_new[index]
    #     print(stock_name)
    tasks.append(task(stock_name,is_new))
    
df = pd.concat(
    joblib.Parallel(n_jobs=5, verbose=10)(
        tasks
    ), axis=1)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Batch computation too fast (0.0060s.) Setting batch_size=2.
[Parallel(n_jobs=5)]: Done   3 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Batch computation too fast (0.0201s.) Setting batch_size=4.
[Parallel(n_jobs=5)]: Done  20 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Batch computation too fast (0.0170s.) Setting batch_size=8.
[Parallel(n_jobs=5)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done  78 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Batch computation too fast (0.0239s.) Setting batch_size=16.
[Parallel(n_jobs=5)]: Done 150 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Batch computation too fast (0.0404s.) Setting batch_size=32.
[Parallel(n_jobs=5)]: Done 342 tasks      | elapsed:    0.1s
[Parallel(n_jobs=5)]: Batch computation too fast (0.0837s.) Setting batch_size=64.
[Pa

In [38]:
df

,000001.SZ,000002.SZ,000004.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,000011.SZ,000012.SZ,...,605507.SH,605555.SH,605566.SH,605567.SH,605577.SH,605580.SH,605588.SH,605589.SH,605598.SH,605599.SH
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2427,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2428,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2429,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2430,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
df.to_csv("new.csv",index=0)

# 全市场选股
1. 剔除60天停牌
2. 剔除60天ST
3. 剔除尾盘涨停
4. 剔除2年新股

In [40]:
suspend_df = pd.read_csv("suspend.csv")
st_df = pd.read_csv("st.csv")
limit_df = pd.read_csv("limit.csv")
new_df = pd.read_csv("new.csv")

In [41]:
df = suspend_df+st_df+limit_df+new_df
df = df.astype(bool)

In [42]:
df

,000001.SZ,000002.SZ,000004.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,000011.SZ,000012.SZ,...,605507.SH,605555.SH,605566.SH,605567.SH,605577.SH,605580.SH,605588.SH,605589.SH,605598.SH,605599.SH
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2427,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2428,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2429,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2430,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [43]:
df.to_csv("all_stock_pool.csv",index=0)

# ST选股
1. 剔除60天停牌
2. 剔除非ST
3. 剔除尾盘涨停
4. 剔除2年新股

In [44]:
# suspend_df = pd.read_csv("suspend.csv")
# not_st_df = pd.read_csv("not_st.csv")
# limit_df = pd.read_csv("limit.csv")
# new_df = pd.read_csv("new.csv")

In [45]:
# df = suspend_df+not_st_df+limit_df+new_df
# df = df.astype(bool)

In [46]:
# df

In [47]:
# df.to_csv("st_stock_pool.csv",index=0)

In [ ]:
# true的地方换掉